# Notebook 04: Multi-Connector Optimization with Explicit Summarization Prompts

## Objective

In this notebook, we'll focus on demonstrating the optimization capabilities of our multi-connector setup. We'll use a simple, explicit summarization prompt to illustrate how the system can optimize the selection of connectors for specific tasks by offloading the summarization skill from ChatGPT to smaller Oobabooga models.

## Prerequisites

- Before proceeding, make sure you have run `00-AI-settings.ipynb` to set up your OpenAI key, since we'll use ChatGPT as our primary connector, and have at least one Oobabooga model running and configured in the `Multiconnector` section of your `settings.json` file according to your VRam capabilities.
- Basic knowledge of the multi-connector pipeline from `03-multiConnector-intro-with-arithmetic-mocks.ipynb`.

## Outline

1. **Setup**
    - Import required libraries
    - Load configuration from settings file
    - Create multicompletion settings
2. **Initialization**
    - Initialize the kernel with multicompletion settings
    - Import summarization skill
3. **Creating a Simple Explicit Summarization Prompt**
    - Define the prompt
    - Create a plan
4. **First Pass: Primary Connector**
    - Execute the plan with the primary connector
    - Analyze the cost and duration
5. **Optimization**
    - Perform tests, evaluation, and optimization
6. **Second Pass: Updated Settings**
    - Re-execute the plan with optimized settings
    - Compare the results
7. **Conclusion**
    - Summary of findings
    - Next steps

## 1. Setup

### 1.1 Import Required Libraries

We'll start with importing the appropriate Nugget packages to load the configuration settings, and to run the Multiconnector itself. 

In [1]:
//Import package for loading hierarchichal settings from settings.json
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4"

// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 0.24.230918.1-preview"
// Import Oobabooga connector package
#r "nuget: MyIA.SemanticKernel.Connectors.AI.Oobabooga"
// Import Multiconnector package
#r "nuget: MyIA.SemanticKernel.Connectors.AI.MultiConnector"

Installed Packages Microsoft.Extensions.Configuration, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Binder, 8.0.0-rc.1.23419.4 Microsoft.Extensions.Configuration.Json, 8.0.0-rc.1.23419.4 Microsoft.SemanticKernel, 0.24.230918.1-preview MyIA.SemanticKernel.Connectors.AI.MultiConnector, 0.33.6 MyIA.SemanticKernel.Connectors.AI.Oobabooga, 0.33.6

### 1.2 Load Settings

We load OpenAi and Multiconnector configuration from the settings file

In [2]:
// Load configuration using builder package
using System.IO;
using Microsoft.Extensions.Configuration;
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.Configuration;

var builder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("config/settings.json", optional: false, reloadOnChange: true);

IConfiguration configuration = builder.Build();

var openAIConfiguration = configuration.GetSection("OpenAI").Get<OpenAIConfiguration>();
var multiOobaboogaConnectorConfiguration = configuration.GetSection("MultiConnector").Get<MultiOobaboogaConnectorConfiguration>();

### 1.3 Set up MultiTextCompletion settings

There are many parameters controlling how the multiconnector will work and perform optimization. We need to create an instance of the corresponding class.

Also, because we'll be measuring costs to perform our optimization, we need to create an creditor object dedicated to that.

For now, we'll stick to the default parameters.

In [ ]:
using MyIA.SemanticKernel.Connectors.AI.MultiConnector;
using System.Text.Json;
using System.Text.Json.Serialization;
using MyIA.SemanticKernel.Connectors.AI.MultiConnector.PromptSettings;

var creditor = new CallRequestCostCreditor();

// The most common settings for a MultiTextCompletion are illustrated below, most of them have default values and are optional
var settings = new MultiTextCompletionSettings()
{
    Creditor = creditor
};

string jsonString = JsonSerializer.Serialize(settings, new JsonSerializerOptions() { WriteIndented = true });
display(jsonString);

## 2. Initialization

With all the settings created, we can now create the semantic kernel that we'll use to run our tests.

### 2.1 Create primary and secondary completions

We'll use the OpenAi configuration to instantiate a text or chat based connector typically using ChatGPT.
Then we can use a helper method from the multicompletion configuration to instantiate all Oobabooga secondary connectors.

In [ ]:
using System.Threading;
using Microsoft.SemanticKernel.AI.TextCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.TextCompletion;

// Creating a cancellation token source to be able to cancel the request
CancellationTokenSource cleanupToken = new();

//Creating the primary connector. We use the OpenAI connector here, either text or chat completion depending on the configuration
 ITextCompletion openAiConnector;

 string testOrChatModelId;
 if (openAIConfiguration.ChatModelId != null)
 {
     testOrChatModelId = openAIConfiguration.ChatModelId;
     openAiConnector = new OpenAIChatCompletion(testOrChatModelId, openAIConfiguration.ApiKey);
 }
 else
 {
     testOrChatModelId = openAIConfiguration.ModelId;
     openAiConnector = new OpenAITextCompletion(testOrChatModelId, openAIConfiguration.ApiKey);
 }

 // Creating the secondary connectors. We use a dedicated helper, but you can create them manually if you want.

 var oobaboogaCompletions = multiOobaboogaConnectorConfiguration.CreateNamedCompletions();

### 2.2 Create Kernel
